# SMS Spam torch ALBERT


ALBERT is a model developed by Google AI, and is a simplified version of BERT that maintains performance while reducing the number of parameters. In the sentence judgment task, it shows performance equal to or better than BERT.

In [47]:
!pip install chardet

    debug = False
    debug2 = False

In [48]:
import numpy as np 
import pandas as pd 
import os
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import matplotlib.pyplot as plt 
import transformers
import random
import chardet
import warnings
warnings.simplefilter('ignore')
scaler = torch.cuda.amp.GradScaler() 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

ModuleNotFoundError: No module named 'torch'

In [ ]:
def random_seed(SEED):
    
    random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    
SEED = 508
random_seed(SEED)

In [44]:
data=pd.read_csv('/kaggle/input/sms-spam-collection-a-more-diverse-dataset/train.csv')
data.columns=['text','targets']
display(data)
classes=sorted(data['targets'].unique().tolist())
print(classes)
class_names=list('01')
N=list(range(len(class_names)))
normal_mapping=dict(zip(class_names,N)) 
reverse_mapping=dict(zip(N,class_names))       
#data['targets']=data['targets'].map(normal_mapping)                   

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/sms-spam-collection-a-more-diverse-dataset/train.csv'

In [ ]:
print(len(data))
data=data.sample(frac=1).reset_index(drop=True)[0:1000]

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
tokenizer = transformers.AlbertTokenizer.from_pretrained("albert-base-v2")

In [ ]:
#tokenizer = transformers.BertTokenizer.from_pretrained("../input/bert-base-uncased")
#tokenizer = transformers.RobertaTokenizer.from_pretrained("roberta-base")

In [ ]:
test_s = train['text'].iloc[0]
result1 = tokenizer.encode_plus(test_s)
tokenizer.decode(result1["input_ids"])

In [ ]:
len(test_s.split(" "))

In [ ]:
result2 = tokenizer.encode_plus(
    test_s,
    add_special_tokens = True, 
    max_length = 8, 
    pad_to_max_length = True, 
    truncation = True 
)

In [ ]:
tokenizer.decode(result2["input_ids"])

In [ ]:
max_sens = 8
train = train.sort_values('targets').reset_index(drop=True)
train["kfold"] = train.index % 5
p_train = train[train["kfold"]!=0].reset_index(drop=True)
p_valid = train[train["kfold"]==0].reset_index(drop=True)
p_test=test.reset_index(drop=True)

'token_type_ids' no need in RoBERTa/DeBERTa

In [ ]:
class BERTDataSet(Dataset):
    
    def __init__(self,sentences,targets):        
        self.sentences = sentences
        self.targets = targets
        
    def __len__(self):        
        return len(self.sentences)
    
    def __getitem__(self,idx):        
        sentence = self.sentences[idx]    
        bert_sens = tokenizer.encode_plus(
                                sentence,
                                add_special_tokens = True, 
                                max_length = max_sens, 
                                pad_to_max_length = True, 
                                return_attention_mask = True)

        ids = torch.tensor(bert_sens['input_ids'], dtype=torch.long)
        mask = torch.tensor(bert_sens['attention_mask'], dtype=torch.long)

        target = torch.tensor(self.targets[idx],dtype=torch.float)
        
        return {
                'ids': ids,
                'mask': mask,
                'targets': target
            }

In [ ]:
train_dataset = BERTDataSet(p_train["text"],p_train['targets'])
valid_dataset = BERTDataSet(p_valid["text"],p_valid['targets'])
test_dataset = BERTDataSet(p_test["text"],p_test['targets'])

train_batch = 16
valid_batch = 32
test_batch = 32

train_dataloader = DataLoader(train_dataset,batch_size=train_batch,shuffle = True,num_workers=8,pin_memory=True)
valid_dataloader = DataLoader(valid_dataset,batch_size=valid_batch,shuffle = False,num_workers=8,pin_memory=True)
test_dataloader = DataLoader(test_dataset,batch_size=test_batch,shuffle = False,num_workers=8,pin_memory=True)

In [ ]:
model = transformers.AlbertForSequenceClassification.from_pretrained("albert-base-v2", num_labels=1)

In [24]:
#model = transformers.RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=1)
#model = transformers.BertForSequenceClassification.from_pretrained("../input/bert-base-uncased",num_labels=1)

In [27]:
model.to(device)
model.train()

NameError: name 'model' is not defined

In [28]:
for a in train_dataloader:
    ids = a["ids"].to(device)
    mask = a["mask"].to(device)
    output = model(ids,mask)
    break

NameError: name 'train_dataloader' is not defined

In [29]:
output = output["logits"].squeeze(-1).shape

NameError: name 'output' is not defined

In [30]:
from transformers import AdamW
LR=2e-5
optimizer = AdamW(model.parameters(), LR,betas=(0.9, 0.999), weight_decay=1e-2) 

NameError: name 'model' is not defined

In [31]:
from transformers import get_linear_schedule_with_warmup
epochs = 20
#if debug:
#    epochs = 1
train_steps = int(len(p_train)/train_batch*epochs)
print(train_steps)
num_steps = int(train_steps*0.1)
scheduler = get_linear_schedule_with_warmup(optimizer, num_steps, train_steps)

NameError: name 'p_train' is not defined

In [32]:
def loss_fn(output,target):
    return torch.sqrt(nn.MSELoss()(output,target))

# def training

In [33]:
def training(
    train_dataloader,
    model,
    optimizer,
    scheduler
):
    
    model.train()
    torch.backends.cudnn.benchmark = True
    allpreds = []
    alltargets = []

    for a in train_dataloader:

        losses = []
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():

            ids = a["ids"].to(device,non_blocking=True)
            mask = a["mask"].to(device,non_blocking=True)

            output = model(ids,mask)
            output = output["logits"].squeeze(-1)
            target = a["targets"].to(device,non_blocking=True)
            loss = loss_fn(output,target)

            losses.append(loss.item())
            allpreds.append(output.detach().cpu().numpy())
            alltargets.append(target.detach().squeeze(-1).cpu().numpy())

        scaler.scale(loss).backward() 
        scaler.step(optimizer) 
        scaler.update() 
        
        del loss 

        scheduler.step() 

    allpreds = np.concatenate(allpreds)
    alltargets = np.concatenate(alltargets)
    losses = np.mean(losses)
    train_rme_loss = np.sqrt(mean_squared_error(alltargets,allpreds))

    return losses,train_rme_loss
#losses,train_rme_loss

# def validating

In [34]:
def validating(valid_dataloader,model):
    
    model.eval()
    allpreds = []
    alltargets = []

    for a in valid_dataloader:
        losses = []
        with torch.no_grad():

            ids = a["ids"].to(device)
            mask = a["mask"].to(device)

            output = model(ids,mask)
            output = output["logits"].squeeze(-1)
            target = a["targets"].to(device)
            loss = loss_fn(output,target)
            losses.append(loss.item())
            allpreds.append(output.detach().cpu().numpy())
            alltargets.append(target.detach().squeeze(-1).cpu().numpy())
            
            del loss

    allpreds = np.concatenate(allpreds)
    alltargets = np.concatenate(alltargets)
    losses = np.mean(losses)
    valid_rme_loss = np.sqrt(mean_squared_error(alltargets,allpreds))

    return allpreds,losses,valid_rme_loss

    if debug2 == False:
        for a in range(epochs):
            for b in train_dataloader:
                break

        losses,train_rme_loss = training(train_dataloader,model,optimizer,scheduler)

        for a in valid_dataloader:
            break

# training and validating

In [35]:
trainlosses = []
vallosses = []
bestscore = None
trainscores = []
validscores = []

for epoch in tqdm(range(epochs)):
    
    print("---------------" + str(epoch) + "start-------------")
    
    trainloss,trainscore = training(train_dataloader,model,optimizer,scheduler)    
    trainlosses.append(trainloss)
    trainscores.append(trainscore)
    
    print("trainscore is " + str(trainscore))
    
    preds,validloss,valscore=validating(valid_dataloader,model)    
    vallosses.append(validloss)
    validscores.append(valscore)
    
    print("valscore is " + str(valscore))
    
    if bestscore is None:
        bestscore = valscore
        
        print("Save first model")
        
        state = {
                        'state_dict': model.state_dict(),
                        'optimizer_dict': optimizer.state_dict(),
                        "bestscore":bestscore
                    }
            
        torch.save(state, "model0.pth")
        
    elif bestscore > valscore:
        
        bestscore = valscore        
        print("found better point")        
        state = {
                        'state_dict': model.state_dict(),
                        'optimizer_dict': optimizer.state_dict(),
                        "bestscore":bestscore
                    }
            
        torch.save(state, "model0.pth")
        
    else:
        pass
    

NameError: name 'tqdm' is not defined

In [36]:
plt.scatter(p_valid['targets'],preds, alpha=0.2)
plt.title('Validation Prediction Result')
plt.xlabel('Actual')
plt.ylabel('Prediction')
plt.show()

x = np.arange(epochs)
plt.title('Validation Losses')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.plot(x,trainlosses)
plt.plot(x,vallosses)
plt.show()

x = np.arange(epochs)
plt.title('Validation Scores')
plt.xlabel('Epoch')
plt.ylabel('Score')
plt.plot(x,trainscores)
plt.plot(x,validscores)
plt.show()

NameError: name 'plt' is not defined

In [37]:
val_true = p_valid['targets']
val_pred = []
for p in preds:
    val_pred+=[round(p,0)]

from sklearn.metrics import classification_report
print(classification_report(val_true,val_pred,target_names=class_names,digits=4))

NameError: name 'p_valid' is not defined

# save model

In [38]:
bestscores = []
bestscores.append(bestscore)

for fold in range(1,5):
    
    # initializing the data
    p_train = train[train["kfold"]!=fold].reset_index(drop=True)
    p_valid = train[train["kfold"]==fold].reset_index(drop=True)

    train_dataset = BERTDataSet(p_train["text"],p_train['targets'])
    valid_dataset = BERTDataSet(p_valid["text"],p_valid['targets'])

    train_dataloader = DataLoader(train_dataset,batch_size=train_batch,shuffle = True,num_workers=4,pin_memory=True)
    valid_dataloader = DataLoader(valid_dataset,batch_size=valid_batch,shuffle = False,num_workers=4,pin_memory=True)

    model = transformers.AlbertForSequenceClassification.from_pretrained("albert-base-v2", num_labels=1)

    model.to(device)
    LR=2e-5
    optimizer = AdamW(model.parameters(), LR,betas=(0.9, 0.999), weight_decay=1e-2) # AdamW optimizer
    train_steps = int(len(p_train)/train_batch*epochs)
    num_steps = int(train_steps*0.1)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_steps, train_steps)

    trainlosses = []
    vallosses = []
    bestscore = None
    trainscores = []
    validscores = []

    for epoch in tqdm(range(epochs)):

        print("---------------" + str(epoch) + "start-------------")

        trainloss,trainscore = training(train_dataloader,model,optimizer,scheduler)
        trainlosses.append(trainloss)
        trainscores.append(trainscore)

        print("trainscore is " + str(trainscore))

        preds,validloss,valscore=validating(valid_dataloader,model)
        vallosses.append(validloss)
        validscores.append(valscore)

        print("valscore is " + str(valscore))

        if bestscore is None:
            bestscore = valscore

            print("Save first model")

            state = {
                            'state_dict': model.state_dict(),
                            'optimizer_dict': optimizer.state_dict(),
                            "bestscore":bestscore
                        }

            torch.save(state, "model" + str(fold) + ".pth") 

        elif bestscore > valscore:
            bestscore = valscore
            print("found better point")

            state = {
                            'state_dict': model.state_dict(),
                            'optimizer_dict': optimizer.state_dict(),
                            "bestscore":bestscore
                        }
            torch.save(state, "model"+ str(fold) + ".pth")

        else:
            pass


    bestscores.append(bestscore)

NameError: name 'train' is not defined

In [39]:
bestscores

[None]

In [40]:
np.mean(bestscores)
print("cv = " + str(np.mean(bestscores)))

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

# def predicting
not use saved models

In [41]:
def predicting(test_dataloader,model):
    
    model.to(device)
    model.eval()   
    allpreds = []
    preds = []
    allvalloss=0

    with torch.no_grad():
        for a in test_dataloader:

            ids = a["ids"].to(device)
            mask = a["mask"].to(device)

            output = model(ids,mask)
            output = output["logits"].squeeze(-1)
            preds.append(output.cpu().numpy())

        preds = np.concatenate(preds)
        allpreds.append(preds)

    return allpreds

# def predicting2
use saved models

    #model initialized
    model = transformers.RobertaForSequenceClassification.from_pretrained("roberta-base",num_labels=1)

    pthes = [os.path.join("./",s) for s in os.listdir("./") if ".pth" in s]

    def predicting2(
        test_dataloader,
        model,
        pthes 
    ):

        allpreds = []    
        for pth in pthes:

            state = torch.load(pth)        
            model.load_state_dict(state["state_dict"])
            model.to(device)
            model.eval()      
            preds = []
            allvalloss=0

            with torch.no_grad():

                for a in test_dataloader:

                    ids = a["ids"].to(device)
                    mask = a["mask"].to(device)

                    output = model(ids,mask)
                    output = output["logits"].squeeze(-1)
                    preds.append(output.cpu().numpy())

                preds = np.concatenate(preds)           
                allpreds.append(preds)

        return allpreds

# predicting

In [42]:
tpreds = predicting(test_dataloader,model)
#tpreds = predicting2(test_dataloader,model,pthes)

NameError: name 'test_dataloader' is not defined

In [43]:
test_true = p_test['targets']
test_pred = []
for p in tpreds[0]:
    test_pred+=[round(p,0)]

from sklearn.metrics import classification_report
print(classification_report(test_true,test_pred,target_names=class_names,digits=4))

NameError: name 'p_test' is not defined